In [1]:
# Code cell intended for imports and global settings.

# Imports
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
from SupportClasses import ModelSupport as ms, EnvironmentSetup as env

# Global values
NUMBER_OF_NODES = 3     # We only support values between 2 and 4 at present.
NUMBER_OF_EPOCHS = 5

# Model specific values
criterion = nn.CrossEntropyLoss()

# Data distributions based on the number of nodes.
data_distribution = env.data_distribution(NUMBER_OF_NODES)

[(0.25, 0.5, 0.25), (0.25, 0.25, 0.5), (0.5, 0.25, 0.25)]


In [2]:
# This code cell is to be used for importing data and setting up the model.
train_loader, validation_loader, test_loader = env.download_mnist()
# Create and load the models. We initiate the model with None as we will update it with the global model in each round.
fed_models = {f"Federated_Model_{i+1}": ms.FederatedModel(train_loader, validation_loader, None)
                for i in range(NUMBER_OF_NODES)}

# Create the baseline, non-federated model.
baseline_model = ms.ConvNet(len(train_loader.dataset.dataset.classes))
# Create the federated model
federated_model = ms.ConvNet(len(train_loader.dataset.dataset.classes))

# Send the models to the CUDA device if it exists.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
baseline_model.to(device=device)
federated_model.to(device=device)

ConvNet(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc1): Linear(in_features=1568, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=10, bias=True)
)

In [3]:
# TODO: We need to get the length of all the datasets

# Here we train a baseline model on all data. No federation as our baseline.
path = 'baseline-model.pt'
optimizer = optim.Adam(baseline_model.parameters())

# We train a new model, if the model does not already exist in memory.
if not os.path.exists(path):
    for epoch in range(NUMBER_OF_EPOCHS):
        start_time = time.time()
        train_loss, train_acc = ms.train(baseline_model, train_loader, optimizer, criterion, device=device)
        valid_loss, valid_acc = ms.test(baseline_model, validation_loader, criterion, device=device)
        end_time = time.time()
        # Get the time to perform non-federated learning
        epoch_mins, epoch_secs = ms.epoch_time(start_time, end_time)

        print(f'Epoch: {epoch+1:02} | Model name: Baseline Model | Epoch time (Baseline Training): {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.5f} | Train Acc: {train_acc*100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.5f} |  Val. Acc: {valid_acc*100:.2f}%')
    torch.save(baseline_model.state_dict(), path)
print("Baseline model training complete.\n\n")

Baseline model training complete.




In [5]:
# Here we train our federated model.
best_valid_loss = float('inf')

for epoch in range(NUMBER_OF_EPOCHS):
    # Perform the computation steps on the individual models
    start_time = time.time()
    for key, fed_model in fed_models.items():
        # Update each model with the global model, before training again.
        fed_model.model.load_state_dict(federated_model.state_dict())
        fed_model.model.to(device=device)

        # Begin training
        optimizer = optim.Adam(fed_model.model.parameters())
        train_loss, train_acc = ms.train(fed_model.model, fed_model.train_loader, optimizer, criterion, device=device)
        valid_loss, valid_acc = ms.test(fed_model.model, fed_model.validation_loader, criterion, device=device)
        print(f'Epoch: {epoch+1:02} | Model name: {key}')
        print(f'\tTrain Loss: {train_loss:.5f} | Train Acc: {train_acc*100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.5f} |  Val. Acc: {valid_acc*100:.2f}%')
    end_time = time.time()
    # Get the time to perform federated learning
    epoch_mins, epoch_secs = ms.epoch_time(start_time, end_time)

    # Average the federated models and combine their weights into the main model.
    len_model = len(train_loader.dataset)
    federated_model.load_state_dict(ms.federated_averaging(fed_models, len_model))
    # Validate this model on a, small balanced validation set
    valid_loss, valid_acc = ms.test(federated_model, validation_loader, criterion, device=device)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        # This will save our best model in case we encounter a drop off during training.
        torch.save(federated_model.state_dict(), 'best-model.pt')

    print(f'Epoch: {epoch+1:02} | Model name: Federated Average | Epoch time (Federated Training): {epoch_mins}m {epoch_secs}s')
    print(f'\t Val. Loss: {valid_loss:.5f} |  Val. Acc: {valid_acc*100:.2f}%')

print("Federated Model training complete.\n\n")

Epoch: 01 | Model name: Federated_Model_1
	Train Loss: 0.00189 | Train Acc: 98.54%
	 Val. Loss: 0.00045 |  Val. Acc: 98.62%
Epoch: 01 | Model name: Federated_Model_2
	Train Loss: 0.00183 | Train Acc: 98.57%
	 Val. Loss: 0.00054 |  Val. Acc: 98.36%
Epoch: 01 | Model name: Federated_Model_3
	Train Loss: 0.00196 | Train Acc: 98.55%
	 Val. Loss: 0.00046 |  Val. Acc: 98.63%
Epoch: 01 | Model name: Federated Average | Epoch time (Federated Training): 0m 58s
	Train Loss: 0.00196 | Train Acc: 98.55%
	 Val. Loss: 0.02016 |  Val. Acc: 33.40%
Epoch: 02 | Model name: Federated_Model_1
	Train Loss: 0.00130 | Train Acc: 99.04%
	 Val. Loss: 0.00045 |  Val. Acc: 98.71%
Epoch: 02 | Model name: Federated_Model_2
	Train Loss: 0.00129 | Train Acc: 99.03%
	 Val. Loss: 0.00041 |  Val. Acc: 98.93%
Epoch: 02 | Model name: Federated_Model_3
	Train Loss: 0.00132 | Train Acc: 98.96%
	 Val. Loss: 0.00051 |  Val. Acc: 98.42%


KeyboardInterrupt: 

In [4]:
# The main testing loop
# Load the model
baseline_model.load_state_dict(torch.load(path))
federated_model.load_state_dict(torch.load('best-model.pt'))

baseline_test_loss, baseline_test_acc = ms.test(baseline_model, test_loader, criterion, device=device)
fed_avg_test_loss, fed_avg_test_acc = ms.test(federated_model, test_loader, criterion, device=device)

print(f'Model name: Baseline | Test Loss: {baseline_test_loss:.3f} | Test Acc: {baseline_test_acc*100:.2f}%')
print(f'Model name: Federated Average | Test Loss: {fed_avg_test_loss:.3f} | Test Acc: {fed_avg_test_acc*100:.2f}%')

Test Loss: 0.000 | Test Acc: 99.01%
